In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.multicomp import pairwise_tukeyhsd, MultiComparison

def load_and_prepare_data(file_path):
    """
    Load and prepare the data for analysis.

    Parameters:
    -----------
    file_path : str
        Path to the CSV file

    Returns:
    --------
    pandas.DataFrame
        Prepared dataframe for analysis
    """
    # Load data - adjust parameters based on your file format
    df = pd.read_csv(file_path)

    # Ensure factor variables are treated as categorical
    categorical_cols = ['Dataset', 'Approach', 'Factors', 'Dims']
    for col in categorical_cols:
        if col in df.columns:
            df[col] = df[col].astype('category')

    # Convert 'Factors' and 'Dims' to string to treat them as categorical factors
    # This is important for the interaction analysis
    if 'Factors' in df.columns:
        df['Factors'] = df['Factors'].astype(str)
    if 'Dims' in df.columns:
        df['Dims'] = df['Dims'].astype(str)

    return df

def fit_linear_mixed_model(df, response_var='Combined_Score'):
    """
    Fit a linear mixed model with Dataset as a random effect and
    Approach, Factors, and Dims as fixed effects.

    Parameters:
    -----------
    df : pandas.DataFrame
        The dataset
    response_var : str
        The response variable to model

    Returns:
    --------
    MixedLM object
        Fitted model
    """
    # Create formula for the mixed model
    # Include main effects and interactions
    formula = (f"{response_var} ~ Approach * Factors * Dims")

    # Fit the mixed effects model with Dataset as a random effect
    model = smf.mixedlm(
        formula=formula,
        data=df,
        groups=df['Dataset']
    )
    result = model.fit()

    return result

def perform_anova(fitted_model, df, response_var='Combined_Score'):
    """
    Perform ANOVA on the model using a Type II approach.

    Parameters:
    -----------
    fitted_model : MixedLM object
        The fitted mixed model
    df : pandas.DataFrame
        The original dataframe
    response_var : str
        The response variable name

    Returns:
    --------
    pandas.DataFrame
        ANOVA table
    """
    # Create a formula for OLS model that matches your mixed model
    formula = f"{response_var} ~ Approach * Factors * Dims"

    # Fit OLS model using the formula API (this will have design_info)
    ols_model = smf.ols(formula=formula, data=df)
    ols_result = ols_model.fit()

    # Perform ANOVA test using the OLS model
    anova_table = sm.stats.anova_lm(ols_result, typ=2)  # Type II ANOVA

    return anova_table


def perform_post_hoc_tests(df, response_var='Combined_Score'):
    """
    Perform post-hoc tests (Tukey's HSD) to compare approaches.
    """
    # Create a multi-comparison object
    mc = MultiComparison(df[response_var], df['Approach'])

    # Perform Tukey's HSD test
    tukey_result = mc.tukeyhsd()

    return mc, tukey_result

def visualize_approach_performance(df, response_var='Combined_Score'):
    """
    Create visualizations to compare approaches' performance.

    Parameters:
    -----------
    df : pandas.DataFrame
        The dataset
    response_var : str
        The response variable

    Returns:
    --------
    matplotlib.figure.Figure
        The created figure
    """
    # Create a figure with multiple subplots
    fig, axes = plt.subplots(2, 2, figsize=(16, 14))

    # 1. Overall performance by approach
    sns.boxplot(x='Approach', y=response_var, data=df, ax=axes[0, 0])
    axes[0, 0].set_title(f'Distribution of {response_var} by Approach')
    axes[0, 0].set_xticklabels(axes[0, 0].get_xticklabels(), rotation=45, ha='right')

    # 2. Performance by approach and dataset
    sns.boxplot(x='Approach', y=response_var, hue='Dataset', data=df, ax=axes[0, 1])
    axes[0, 1].set_title(f'{response_var} by Approach and Dataset')
    axes[0, 1].set_xticklabels(axes[0, 1].get_xticklabels(), rotation=45, ha='right')
    axes[0, 1].legend(loc='upper right', bbox_to_anchor=(1.25, 1))

    # 3. Performance by approach and factors
    sns.boxplot(x='Approach', y=response_var, hue='Factors', data=df, ax=axes[1, 0])
    axes[1, 0].set_title(f'{response_var} by Approach and Factors')
    axes[1, 0].set_xticklabels(axes[1, 0].get_xticklabels(), rotation=45, ha='right')

    # 4. Performance by approach and dimensions
    sns.boxplot(x='Approach', y=response_var, hue='Dims', data=df, ax=axes[1, 1])
    axes[1, 1].set_title(f'{response_var} by Approach and Dimensions')
    axes[1, 1].set_xticklabels(axes[1, 1].get_xticklabels(), rotation=45, ha='right')

    plt.tight_layout()
    return fig

def analyze_interaction_effects(df, response_var='Combined_Score'):
    """
    Analyze and visualize interaction effects between factors.

    Parameters:
    -----------
    df : pandas.DataFrame
        The dataset
    response_var : str
        The response variable

    Returns:
    --------
    matplotlib.figure.Figure
        The created figure
    """
    # Create a figure with interaction plots
    fig, axes = plt.subplots(2, 2, figsize=(16, 14))

    # 1. Approach x Factors interaction
    for approach in df['Approach'].unique():
        subset = df[df['Approach'] == approach]
        means = subset.groupby('Factors', observed=True)[response_var].mean()
        axes[0, 0].plot(means.index, means.values, marker='o', label=approach)

    axes[0, 0].set_title('Interaction: Approach x Factors')
    axes[0, 0].set_xlabel('Number of Factors')
    axes[0, 0].set_ylabel(response_var)
    axes[0, 0].legend(loc='upper right', bbox_to_anchor=(1.25, 1))

    # 2. Approach x Dims interaction
    for approach in df['Approach'].unique():
        subset = df[df['Approach'] == approach]
        means = subset.groupby('Dims', observed=True)[response_var].mean()
        axes[0, 1].plot(means.index, means.values, marker='o', label=approach)

    axes[0, 1].set_title('Interaction: Approach x Dimensions')
    axes[0, 1].set_xlabel('Number of Dimensions')
    axes[0, 1].set_ylabel(response_var)

    # 3. Dataset x Approach interaction
    for dataset in df['Dataset'].unique():
        subset = df[df['Dataset'] == dataset]
        means = subset.groupby('Approach', observed=True)[response_var].mean()
        axes[1, 0].plot(means.index, means.values, marker='o', label=dataset)

    axes[1, 0].set_title('Interaction: Dataset x Approach')
    axes[1, 0].set_xlabel('Approach')
    axes[1, 0].set_ylabel(response_var)
    axes[1, 0].set_xticks(range(len(df['Approach'].unique())))
    axes[1, 0].set_xticklabels(df['Approach'].unique(), rotation=45, ha='right')
    axes[1, 0].legend(loc='upper right', bbox_to_anchor=(1.25, 1))

    # 4. Factors x Dims interaction
    for factor in df['Factors'].unique():
        subset = df[df['Factors'] == factor]
        means = subset.groupby('Dims', observed=True)[response_var].mean()
        axes[1, 1].plot(means.index, means.values, marker='o', label=f'Factors={factor}')

    axes[1, 1].set_title('Interaction: Factors x Dimensions')
    axes[1, 1].set_xlabel('Number of Dimensions')
    axes[1, 1].set_ylabel(response_var)
    axes[1, 1].legend()

    plt.tight_layout()
    return fig

def analyze_dataset_variability(df, response_var='Combined_Score'):
    """
    Analyze variability across datasets and how it influences approach performance.

    Parameters:
    -----------
    df : pandas.DataFrame
        The dataset
    response_var : str
        The response variable

    Returns:
    --------
    tuple
        Results dictionary and figure
    """
    # Create a figure for visualizing dataset variability
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))

    # 1. Distribution of response variable by dataset
    sns.boxplot(x='Dataset', y=response_var, data=df, ax=axes[0])
    axes[0].set_title(f'Distribution of {response_var} by Dataset')

    # 2. Heatmap showing average performance of approaches across datasets
    pivot_table = df.pivot_table(
        values=response_var,
        index='Dataset',
        columns='Approach',
        aggfunc='mean',
        observed=True
    )

    # Create heatmap
    sns.heatmap(pivot_table, annot=True, cmap='viridis', ax=axes[1], fmt=".2f")
    axes[1].set_title('Mean Performance by Dataset and Approach')

    plt.tight_layout()

    # Calculate dataset variability metrics
    dataset_means = df.groupby('Dataset', observed=True)[response_var].mean().to_dict()
    dataset_vars = df.groupby('Dataset', observed=True)[response_var].var().to_dict()

    # Find best approach for each dataset
    best_approaches = {}
    for dataset in df['Dataset'].unique():
        subset = df[df['Dataset'] == dataset]
        best_approach = subset.groupby('Approach', observed=True)[response_var].mean().idxmax()
        best_approaches[dataset] = best_approach

    # Calculate consistency of approach performance across datasets
    approach_consistency = {}
    for approach in df['Approach'].unique():
        subset = df[df['Approach'] == approach]
        # Calculate coefficient of variation across datasets
        means_by_dataset = subset.groupby('Dataset')[response_var].mean()
        cv = means_by_dataset.std() / means_by_dataset.mean()
        approach_consistency[approach] = {
            'mean': means_by_dataset.mean(),
            'std': means_by_dataset.std(),
            'cv': cv,  # Lower CV means more consistent performance
            'ranks': []  # Will store rank of approach in each dataset
        }

    # Calculate approach rankings within each dataset
    for dataset in df['Dataset'].unique():
        subset = df[df['Dataset'] == dataset]
        ranks = subset.groupby('Approach')[response_var].mean().rank(ascending=False)

        for approach in ranks.index:
            approach_consistency[approach]['ranks'].append(ranks[approach])

    # Calculate average rank for each approach
    for approach in approach_consistency:
        approach_consistency[approach]['avg_rank'] = np.mean(approach_consistency[approach]['ranks'])

    results = {
        'dataset_means': dataset_means,
        'dataset_vars': dataset_vars,
        'best_approaches': best_approaches,
        'approach_consistency': approach_consistency
    }

    return results, fig

def main(file_path, response_var='Combined_Score'):
    """
    Main function to run the complete analysis.
    """
    # Load and prepare data
    print("Loading and preparing data...")
    df = load_and_prepare_data(file_path)

    # Fit linear mixed model
    print("Fitting linear mixed model...")
    lmm_result = fit_linear_mixed_model(df, response_var)
    print("\nLinear Mixed Model Summary:")
    print(lmm_result.summary())

    # Perform ANOVA
    print("\nPerforming ANOVA...")
    anova_result = perform_anova(lmm_result, df, response_var)
    print("\nANOVA Results:")
    print(anova_result)

    # --- Perform post-hoc tests ---
    mc, tukey_result = perform_post_hoc_tests(df, response_var)
    print("\nTukey's HSD Results:")
    print(tukey_result)

    # --- FIXED TUKEY HANDLING ---
    # 1) force the group names into a NumPy array
    group_names = np.array(list(tukey_result.groupsunique))

    # 2) tukey_result._multicomp.pairindices is a tuple of two index arrays
    idx1, idx2 = tukey_result._multicomp.pairindices

    # 3) build the DataFrame
    tukey_df = pd.DataFrame({
        'group1':   group_names[idx1],
        'group2':   group_names[idx2],
        'meandiff': tukey_result.meandiffs,
        'p-adj':    tukey_result.pvalues,
        'reject':   tukey_result.reject
    })

    # 4) filter to only those pairs where reject == True
    sig_pairs = tukey_df[tukey_df['reject']]
    # ------------------------------

    # Create visualizations
    print("\nCreating visualizations...")
    performance_fig = visualize_approach_performance(df, response_var)
    interaction_fig = analyze_interaction_effects(df, response_var)
    dataset_results, dataset_fig = analyze_dataset_variability(df, response_var)

    # Compile results
    results = {
        'data':            df,
        'lmm_result':      lmm_result,
        'anova_result':    anova_result,
        'tukey_result':    tukey_result,
        'sig_pairs':       sig_pairs,
        'dataset_analysis': dataset_results,
        'figures': {
            'performance': performance_fig,
            'interaction': interaction_fig,
            'dataset':     dataset_fig
        }
    }

    # Print key findings
    print("\nKey Findings:")
    print("=============")

    # 1. Significant Factors
    print("\n1. Significant Factors:")
    sig_facts = anova_result[anova_result['PR(>F)'] < 0.05].index.tolist()
    if sig_facts:
        for f in sig_facts:
            print(f"  - {f}: p = {anova_result.loc[f,'PR(>F)']:.4f}")
    else:
        print("  - None at α = 0.05")

    # 2. Approach comparisons
    print("\n2. Approach Comparison:")
    if not sig_pairs.empty:
        print("  Significant pairwise differences:")
        for _, row in sig_pairs.iterrows():
            print(f"  - {row['group1']} vs {row['group2']}: p‑adj = {row['p-adj']:.4f}")
    else:
        print("  - No significant differences between approaches at α = 0.05")

    # 3. Best approach by dataset
    print("\n3. Best Approach by Dataset:")
    for ds, appr in dataset_results['best_approaches'].items():
        mean_score = df[(df['Dataset']==ds)&(df['Approach']==appr)][response_var].mean()
        print(f"  - {ds}: {appr} (mean {response_var}={mean_score:.4f})")

    # 4. Factors & Dims effects
    print("\n4. Effects of Factors and Dimensions:")
    for fac, m in df.groupby('Factors', observed=True)[response_var].mean().items():
        print(f"  - Factors={fac}: {m:.4f}")
    for dim, m in df.groupby('Dims', observed=True)[response_var].mean().items():
        print(f"  - Dims={dim}:    {m:.4f}")

    # 5. Dataset variability
    print("\n5. Dataset Variability:")
    vars_ = dataset_results['dataset_vars']
    most = max(vars_, key=vars_.get)
    least= min(vars_, key=vars_.get)
    print(f"  - Most variable: {most} (var={vars_[most]:.4f})")
    print(f"  - Least variable: {least} (var={vars_[least]:.4f})")

    # 6. Approach consistency
    print("\n6. Approach Consistency:")
    for appr, metr in sorted(dataset_results['approach_consistency'].items(),
                              key=lambda kv: kv[1]['avg_rank']):
        print(f"  - {appr}: avg rank={metr['avg_rank']:.2f}, CV={metr['cv']:.4f}")

    return results


# Example of how to run the analysis
if __name__ == "__main__":
    # Replace with your actual file path
    file_path = "C:/Users/taiku/Documents/universal_K_data.csv"

    # Run the analysis
    results = main(file_path)

    # Show the figures
    plt.show()

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.multicomp import pairwise_tukeyhsd, MultiComparison
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Load the data
def load_data(file_path):
    """Load data from CSV file and convert columns to appropriate types"""
    df = pd.read_csv(file_path)

    # Convert factors and dims to string to treat as categorical variables
    df['Factors'] = df['Factors'].astype(str)
    df['Dims'] = df['Dims'].astype(str)

    return df

# 2. Fit the Linear Mixed Model
def fit_lmm(df, response_var='Combined Score'):
    """
    Fit a linear mixed model with Dataset as random effect
    and Approach, Factors, and Dims as fixed effects
    """
    # Formula for the model with main effects and interactions
    formula = f"{response_var} ~ Approach * Factors * Dims"

    # Fit LMM with Dataset as random effect
    model = smf.mixedlm(
        formula=formula,
        data=df,
        groups=df['Dataset']
    )

    # Use robust standard errors
    result = model.fit(reml=True)

    return result

# 3. Perform ANOVA
def run_anova(lmm_result):
    """Perform ANOVA on the fitted LMM to test significance of fixed effects"""
    # Extract design matrix
    X = lmm_result.model.exog

    # Get names of fixed effects
    names = lmm_result.model.exog_names

    # Fit OLS model for ANOVA testing
    ols_model = sm.OLS(lmm_result.model.endog, X)
    ols_result = ols_model.fit()

    # Run ANOVA
    anova_table = anova_lm(ols_result)

    return anova_table

# 4. Post-hoc tests
def post_hoc_analysis(df, response_var='Combined Score'):
    """Perform post-hoc Tukey's HSD tests for approach comparisons"""
    # Post-hoc test for Approach
    approach_mc = MultiComparison(df[response_var], df['Approach'])
    approach_result = approach_mc.tukeyhsd()

    # Post-hoc test for Factors
    factor_mc = MultiComparison(df[response_var], df['Factors'])
    factor_result = factor_mc.tukeyhsd()

    # Post-hoc test for Dims
    dim_mc = MultiComparison(df[response_var], df['Dims'])
    dim_result = dim_mc.tukeyhsd()

    return {
        'approach': (approach_mc, approach_result),
        'factors': (factor_mc, factor_result),
        'dims': (dim_mc, dim_result)
    }

# 5. Visualize interactions
def plot_interactions(df, response_var='Combined Score'):
    """Create interaction plots to visualize relationship between factors"""
    # Set up the figure
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))

    # Approach x Dataset interaction
    avg_by_approach_dataset = df.groupby(['Dataset', 'Approach'], observed=True)[response_var].mean().unstack()
    avg_by_approach_dataset.plot(marker='o', ax=axes[0, 0])
    axes[0, 0].set_title('Interaction: Dataset x Approach')
    axes[0, 0].set_xlabel('Dataset')
    axes[0, 0].set_ylabel(response_var)

    # Approach x Factors interaction
    avg_by_approach_factors = df.groupby(['Factors', 'Approach'], observed=True)[response_var].mean().unstack()
    avg_by_approach_factors.plot(marker='o', ax=axes[0, 1])
    axes[0, 1].set_title('Interaction: Factors x Approach')
    axes[0, 1].set_xlabel('Number of Factors')
    axes[0, 1].set_ylabel(response_var)

    # Approach x Dims interaction
    avg_by_approach_dims = df.groupby(['Dims', 'Approach'], observed=True)[response_var].mean().unstack()
    avg_by_approach_dims.plot(marker='o', ax=axes[1, 0])
    axes[1, 0].set_title('Interaction: Dimensions x Approach')
    axes[1, 0].set_xlabel('Number of Dimensions')
    axes[1, 0].set_ylabel(response_var)

    # Factors x Dims interaction
    factors_dims_pivot = df.pivot_table(
        values=response_var,
        index='Factors',
        columns='Dims',
        aggfunc='mean'
    )
    sns.heatmap(factors_dims_pivot, annot=True, cmap='viridis', ax=axes[1, 1])
    axes[1, 1].set_title('Interaction: Factors x Dimensions')

    plt.tight_layout()
    return fig

# 6. Analyze dataset variability
def analyze_dataset_effects(df, response_var='Combined Score'):
    """Analyze how dataset affects performance of different approaches"""
    # Create figure
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))

    # Plot mean scores by approach for each dataset
    sns.boxplot(x='Approach', y=response_var, hue='Dataset', data=df, ax=axes[0])
    axes[0].set_title('Performance by Approach and Dataset')
    axes[0].set_xticklabels(axes[0].get_xticklabels(), rotation=45, ha='right')

    # Create heatmap of approach performance by dataset
    heatmap_data = df.pivot_table(
        values=response_var,
        index='Dataset',
        columns='Approach',
        aggfunc='mean'
    )

    # Find the best approach for each dataset (highlight cells)
    best_mask = heatmap_data.apply(lambda x: x == x.max(), axis=1)

    # Plot heatmap
    sns.heatmap(heatmap_data, annot=True, fmt='.3f', cmap='viridis', ax=axes[1])
    axes[1].set_title('Mean Performance by Dataset and Approach')

    plt.tight_layout()

    # Calculate best approach for each dataset
    best_approaches = {}
    for dataset in df['Dataset'].unique():
        dataset_df = df[df['Dataset'] == dataset]
        approach_means = dataset_df.groupby('Approach')[response_var].mean()
        best_approach = approach_means.idxmax()
        best_approaches[dataset] = (best_approach, approach_means[best_approach])

    return best_approaches, fig

# Main function to run the analysis
def main(file_path, response_var='Combined Score'):
    """Run the complete analysis and return results"""
    print("Loading data...")
    df = load_data(file_path)

    print("\nFitting Linear Mixed Model...")
    lmm_result = fit_lmm(df, response_var)
    print(lmm_result.summary())

    print("\nPerforming ANOVA to test significance of fixed effects...")
    anova_result = run_anova(lmm_result)
    print("\nANOVA Results:")
    print(anova_result)

    print("\nPerforming post-hoc tests (Tukey's HSD)...")
    post_hoc_results = post_hoc_analysis(df, response_var)

    print("\nTukey's HSD Results for Approach:")
    print(post_hoc_results['approach'][1])

    print("\nTukey's HSD Results for Factors:")
    print(post_hoc_results['factors'][1])

    print("\nTukey's HSD Results for Dimensions:")
    print(post_hoc_results['dims'][1])

    print("\nCreating interaction plots...")
    interaction_fig = plot_interactions(df, response_var)

    print("\nAnalyzing dataset effects...")
    best_approaches, dataset_fig = analyze_dataset_effects(df, response_var)

    print("\nBest approach for each dataset:")
    for dataset, (approach, score) in best_approaches.items():
        print(f"  - {dataset}: {approach} (score: {score:.4f})")

    # Summarize main findings
    print("\nSummary of Main Findings:")
    print("-------------------------")

    # Check if Approach is significant
    p_value_approach = anova_result.loc['Approach', 'PR(>F)'] if 'Approach' in anova_result.index else None
    if p_value_approach is not None and p_value_approach < 0.05:
        print(f"1. Approach has a significant effect on {response_var} (p = {p_value_approach:.4f})")

        # Count significant pairwise comparisons
        approach_result = post_hoc_results['approach'][1]
        sig_pairs = sum(approach_result.reject)

        if sig_pairs > 0:
            print(f"   - {sig_pairs} significant pairwise differences found among approaches")

            # Print top 5 most significant differences
            pair_indices = approach_result._multicomp.pairindices
            approaches = approach_result.groupsunique

            # Get pairs and their p-values
            pair_data = []
            for i, (idx1, idx2) in enumerate(pair_indices):
                if approach_result.reject[i]:
                    app1 = approaches[idx1]
                    app2 = approaches[idx2]
                    meandiff = approach_result.meandiffs[i]
                    pvalue = approach_result.pvalues[i]
                    pair_data.append((app1, app2, meandiff, pvalue))

            # Sort by p-value (ascending)
            pair_data.sort(key=lambda x: x[3])

            # Print top 5 most significant differences
            print("   - Top significant pairwise differences:")
            for i, (app1, app2, meandiff, pvalue) in enumerate(pair_data[:5]):
                print(f"     {app1} vs {app2}: diff = {meandiff:.4f}, p-adj = {pvalue:.4f}")
    else:
        print(f"1. Approach does NOT have a significant effect on {response_var}")

    # Check if Factors is significant
    p_value_factors = anova_result.loc['Factors', 'PR(>F)'] if 'Factors' in anova_result.index else None
    if p_value_factors is not None and p_value_factors < 0.05:
        print(f"2. Number of factors has a significant effect on {response_var} (p = {p_value_factors:.4f})")

        # Show mean scores for different numbers of factors
        factor_means = df.groupby('Factors')[response_var].mean()
        for factor, mean in factor_means.items():
            print(f"   - Factors={factor}: mean {response_var} = {mean:.4f}")
    else:
        print(f"2. Number of factors does NOT have a significant effect on {response_var}")

    # Check if Dims is significant
    p_value_dims = anova_result.loc['Dims', 'PR(>F)'] if 'Dims' in anova_result.index else None
    if p_value_dims is not None and p_value_dims < 0.05:
        print(f"3. Number of dimensions has a significant effect on {response_var} (p = {p_value_dims:.4f})")

        # Show mean scores for different numbers of dimensions
        dims_means = df.groupby('Dims')[response_var].mean()
        for dim, mean in dims_means.items():
            print(f"   - Dims={dim}: mean {response_var} = {mean:.4f}")
    else:
        print(f"3. Number of dimensions does NOT have a significant effect on {response_var}")

    # Check interaction effects
    interaction_terms = [term for term in anova_result.index if '*' in term]
    significant_interactions = [term for term in interaction_terms
                               if anova_result.loc[term, 'PR(>F)'] < 0.05]

    if significant_interactions:
        print("4. Significant interaction effects found:")
        for term in significant_interactions:
            p_value = anova_result.loc[term, 'PR(>F)']
            print(f"   - {term}: p = {p_value:.4f}")
    else:
        print("4. No significant interaction effects found")

    # Dataset variability
    print("5. Dataset variability insights:")
    dataset_means = df.groupby('Dataset')[response_var].mean().sort_values(ascending=False)
    dataset_vars = df.groupby('Dataset')[response_var].var().sort_values(ascending=False)

    print("   - Dataset performance ranking:")
    for dataset, mean in dataset_means.items():
        print(f"     {dataset}: mean {response_var} = {mean:.4f}")

    print("   - Dataset variability ranking:")
    for dataset, var in dataset_vars.items():
        print(f"     {dataset}: variance = {var:.4f}")

    # Return all results
    results = {
        'lmm_result': lmm_result,
        'anova_result': anova_result,
        'post_hoc_results': post_hoc_results,
        'best_approaches': best_approaches,
        'figures': {
            'interaction_fig': interaction_fig,
            'dataset_fig': dataset_fig
        }
    }

    return results

# Run the analysis if script is executed directly
if __name__ == "__main__":
    file_path = "data.csv"  # Update this to your actual file path
    results = main(file_path)
    plt.show()  # Display all figures